In [1]:
%load_ext autoreload
%autoreload 2

In [85]:
#import mutagen
import os
import pickle
import random
import wave
import numpy as np
from scipy.io.wavfile import read as read_wav
from scipy.io.wavfile import write as write_wav
import librosa
import librosa.display
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from sklearn.metrics import plot_confusion_matrix, confusion_matrix, precision_recall_fscore_support

import matplotlib.pyplot as plt

import utils
import classifier
import multi_classifier as mclass
import irmasTrainUtils as trainUtils
import irmasTestUtils as testUtils

In [3]:
# Load dataset

df_train = utils.load_pickled_dataset("../train.pkl")
df_valid = utils.load_pickled_dataset("../validation.pkl")
df_train = df_train.sample(frac=1)
df_valid = df_valid.sample(frac=1)

In [4]:
# Verify the accuracies are as expected

test = classifier.make_model()
test.load_weights("weights")
print(classifier.get_accuracy(test, df_train))
print(classifier.get_accuracy(test, df_valid))

0.7036668325866932
0.6047197640117994


In [29]:
# Get the list of true labels

true = df_valid['label'].apply(lambda x: np.argmax(x)).values

In [ ]:
# Get the list of predictions

SAMPLE_SHAPE = (128,259)
x, y = np.stack(df_valid.data), np.stack(df_valid.label)
x = x.reshape((-1,SAMPLE_SHAPE[0],SAMPLE_SHAPE[1],2))
preds = np.argmax(test.predict(x), axis=1).reshape(x.shape[0], 1)
preds = preds.reshape(preds.shape[0],)

In [ ]:
true

In [33]:
preds

array([6, 3, 6, ..., 7, 5, 6])

In [56]:
# Generate a confusion matrix

conf_matrix = confusion_matrix(true, preds, normalize=None)

In [57]:
print (conf_matrix)

[[ 29   1   3  12   4   0  11   4   0   8   0]
 [  0  51   6   5   2   0  22   8   2   4   0]
 [  2  10  40   8   3   6  23   0   0   0   0]
 [  0   2   0  89   2   4  15   0   0   0   0]
 [  0   2   2  23  68   8  36   0   2   3   0]
 [  0   0   2   5   7  88  36   2   2   2   0]
 [  0   0   0   8   0   2 146   0   0   0   0]
 [  6   4  10  10   2   0  16  59   6   4   1]
 [  0   5   4   0   0   4  21  18  64   6   0]
 [ 14   5   9   8  12   0  11  10   1  62   0]
 [  0   0   2  12  18   0   8   0   0   0 124]]


In [55]:
conf_matrix[0]

array([29,  1,  3, 12,  4,  0, 11,  4,  0,  8,  0,  0])

In [82]:
# Generate a normalized confusion matrix

conf_matrix_norm = confusion_matrix(true, preds, normalize='true')
conf_matrix_norm = np.round(conf_matrix_norm, 2)

In [62]:
print(np.round(conf_matrix_norm, 2))

[[0.4  0.01 0.04 0.17 0.06 0.   0.15 0.06 0.   0.11 0.  ]
 [0.   0.51 0.06 0.05 0.02 0.   0.22 0.08 0.02 0.04 0.  ]
 [0.02 0.11 0.43 0.09 0.03 0.07 0.25 0.   0.   0.   0.  ]
 [0.   0.02 0.   0.79 0.02 0.04 0.13 0.   0.   0.   0.  ]
 [0.   0.01 0.01 0.16 0.47 0.06 0.25 0.   0.01 0.02 0.  ]
 [0.   0.   0.01 0.03 0.05 0.61 0.25 0.01 0.01 0.01 0.  ]
 [0.   0.   0.   0.05 0.   0.01 0.94 0.   0.   0.   0.  ]
 [0.05 0.03 0.08 0.08 0.02 0.   0.14 0.5  0.05 0.03 0.01]
 [0.   0.04 0.03 0.   0.   0.03 0.17 0.15 0.52 0.05 0.  ]
 [0.11 0.04 0.07 0.06 0.09 0.   0.08 0.08 0.01 0.47 0.  ]
 [0.   0.   0.01 0.07 0.11 0.   0.05 0.   0.   0.   0.76]]


In [74]:
# Convert to a dataframe

labels = ["cel", "cla", "flu", "gac", "gel", "org", "pia", "sax", "tru", "vio", "voi"]
res = pd.DataFrame (conf_matrix, columns = labels, index = labels)

In [75]:
res

,cel,cla,flu,gac,gel,org,pia,sax,tru,vio,voi
cel,29,1,3,12,4,0,11,4,0,8,0
cla,0,51,6,5,2,0,22,8,2,4,0
flu,2,10,40,8,3,6,23,0,0,0,0
gac,0,2,0,89,2,4,15,0,0,0,0
gel,0,2,2,23,68,8,36,0,2,3,0
org,0,0,2,5,7,88,36,2,2,2,0
pia,0,0,0,8,0,2,146,0,0,0,0
sax,6,4,10,10,2,0,16,59,6,4,1
tru,0,5,4,0,0,4,21,18,64,6,0
vio,14,5,9,8,12,0,11,10,1,62,0


In [83]:
res_norm = pd.DataFrame(conf_matrix_norm, columns = labels, index = labels)

In [84]:
res_norm

,cel,cla,flu,gac,gel,org,pia,sax,tru,vio,voi
cel,0.40,0.01,0.04,0.17,0.06,0.00,0.15,0.06,0.00,0.11,0.00
cla,0.00,0.51,0.06,0.05,0.02,0.00,0.22,0.08,0.02,0.04,0.00
flu,0.02,0.11,0.43,0.09,0.03,0.07,0.25,0.00,0.00,0.00,0.00
gac,0.00,0.02,0.00,0.79,0.02,0.04,0.13,0.00,0.00,0.00,0.00
gel,0.00,0.01,0.01,0.16,0.47,0.06,0.25,0.00,0.01,0.02,0.00
org,0.00,0.00,0.01,0.03,0.05,0.61,0.25,0.01,0.01,0.01,0.00
pia,0.00,0.00,0.00,0.05,0.00,0.01,0.94,0.00,0.00,0.00,0.00
sax,0.05,0.03,0.08,0.08,0.02,0.00,0.14,0.50,0.05,0.03,0.01
tru,0.00,0.04,0.03,0.00,0.00,0.03,0.17,0.15,0.52,0.05,0.00
vio,0.11,0.04,0.07,0.06,0.09,0.00,0.08,0.08,0.01,0.47,0.00


In [101]:
# Generate the metrics for our model (precision, recall, F1 Score)

metrics = pd.DataFrame(
    np.round(precision_recall_fscore_support (true, preds), 2), 
        columns = labels, 
        index = ["Precision", "Recall", "F-1 Score", "Count"]
)

In [102]:
metrics

,cel,cla,flu,gac,gel,org,pia,sax,tru,vio,voi
Precision,0.57,0.64,0.51,0.49,0.58,0.79,0.42,0.58,0.83,0.70,0.99
Recall,0.40,0.51,0.43,0.79,0.47,0.61,0.94,0.50,0.52,0.47,0.76
F-1 Score,0.47,0.57,0.47,0.61,0.52,0.69,0.58,0.54,0.64,0.56,0.86
Count,72.00,100.00,92.00,112.00,144.00,144.00,156.00,118.00,122.00,132.00,164.00


In [103]:
# Save to CSV output

metrics.to_csv ("metrics_single_classifier.csv")

In [104]:
# Save to CSV output

res.to_csv("results_single_classifier.csv")